In [1]:
# Télécharger les ressources necéssaires pour utiliser le script
#!pip install python-binance
#!pip install pytz
#from analysis_functions import find_imbalances_after_rise, find_imbalance_after_fall

import pip, os, pytz, pandas as pd
from datetime import date, timedelta, datetime
from market_imbalance import MarketImbalance, MarketCandle, create_market_candle, find_imbalances_after_fall, find_imbalances_after_rise, get_fullfilled_imbalances_after_fall, get_fullfilled_imbalances_after_rise
from interact_with_binance import Trade, fetch_ohlcv_as_df, fetch_ohlcv, adjust_timestamps_to_local
from binance.client import Client



In [2]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)
#server_time = pd.to_datetime(client.get_server_time()['serverTime'], unit='ms')
#my_time = datetime.now(pytz.timezone('Europe/Paris'))

# Récupérer l'historique des prix pour une paire donnée depuis son listing sur la plateforme
# df = fetch_ohlcv_as_df(client, symbol = "BTCUSDT", interval=Client.KLINE_INTERVAL_1MINUTE, from_date= "31 mars 2024") #"1 Jan, 2015"
# df = adjust_timestamps_to_local(df, 'timestamp')

In [4]:
historical_candles_data = fetch_ohlcv(client, symbol = "BTCUSDT", interval=Client.KLINE_INTERVAL_1MINUTE, from_date= "31 mars 2024") #"1 Jan, 2015"

# Créer des objets MarketCandle à partir de la liste de listes
historical_candles_object = [MarketCandle(*candle) for candle in historical_candles_data]

# Afficher les objets MarketCandle créés
print(len(historical_candles_object))
print()
for candle in historical_candles_object:
    print(candle.timestamp, candle.open_price, candle.high_price, candle.low_price, candle.close_price)
    

3427

1711843200000 69582.17000000 69618.90000000 69562.99000000 69618.89000000
1711843260000 69618.89000000 69632.05000000 69606.00000000 69632.05000000
1711843320000 69632.04000000 69632.05000000 69607.10000000 69607.10000000
1711843380000 69607.10000000 69617.14000000 69607.10000000 69617.13000000
1711843440000 69617.13000000 69632.95000000 69617.13000000 69632.94000000
1711843500000 69632.94000000 69685.21000000 69632.94000000 69667.29000000
1711843560000 69667.28000000 69667.29000000 69640.00000000 69660.18000000
1711843620000 69660.18000000 69680.00000000 69660.17000000 69680.00000000
1711843680000 69679.99000000 69713.00000000 69679.99000000 69712.99000000
1711843740000 69713.00000000 69713.00000000 69690.00000000 69690.00000000
1711843800000 69690.01000000 69708.18000000 69690.00000000 69708.17000000
1711843860000 69708.17000000 69708.18000000 69708.17000000 69708.18000000
1711843920000 69708.17000000 69708.18000000 69701.20000000 69702.04000000
1711843980000 69702.03000000 697

Les scripts ci-dessous permettent de vérifier les imbalances trouvés en les comparants avec les données de tradingview

In [ ]:
imbalances_after_fall = find_imbalances_after_fall(df)
fullfilled_imbalances_after_fall = get_fullfilled_imbalances_after_fall(imbalances_after_fall, df)
# for imb in fullfilled_imbalances_after_fall[-10:] : 
#    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

In [ ]:
imbalances_after_rise = find_imbalances_after_rise(df)
fullfilled_imbalances_after_rise = get_fullfilled_imbalances_after_rise(imbalances_after_rise, df)
# for imb in fullfilled_imbalances_after_rise[-10:] : 
#    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

Les scripts ci-dessous traitent de l'analyse des imbalances comblés

In [ ]:
# Percentage of gap filled after fall
percentage_filled_after_fall = (len(fullfilled_imbalances_after_fall) / len(imbalances_after_fall)) * 100
print(f'% of filled gap after fall = {percentage_filled_after_fall} with {len(fullfilled_imbalances_after_fall)} fullfilled imbalances on {len(imbalances_after_fall)} in total')

# Percentage of gap filled after rise
percentage_filled_after_rise = (len(fullfilled_imbalances_after_rise) / len(imbalances_after_rise)) * 100
print(f'% of filled gap after rise = {percentage_filled_after_rise} with {len(fullfilled_imbalances_after_rise)} fullfilled imbalances on {len(imbalances_after_rise)} in total')

Les scripts ci-dessous traitent de l'identification des imbalances non comblés, sous entendu des trades à prendre.
Notre objectif ici est :
1. Récupérer la liste des imbalances non comblés après une baisse
2. Récupérer la liste des imbalances non comblés après une hausse
3. Calculer la somme des imbalances incomblés
4. si le prix du marché pénètre une zone imbalance, alors ouvrir un trade pour l'imbalance concerné.
5. si le prix du marché pénètre la zone et déclenche notre stop-loss, alors rouvrir un trade.
6. si le prix du marché pénètre la zone et ressort de la zone sans déclencher notre stop-loss, alors conserver uniquement le trade déjà ouvert.

Notre stratégie n'est pas d'ouvrir un trade par imbalance non comblé. Notre stratégie est d'ouvrir un trade à chaque fois que le prix du marché pénètre une zone d'imbalance. Si le prix rentre dans la zone et déclenche notre stop-loss, nous devons réouvrir un trade lorsque le prix re-rentre dans la zone d'imbalance.


In [ ]:
# Fonctions pour identifier les imbalances non comblés après une baisse et après une hausse
unfilled_imbalance_after_fall = []
unfilled_imbalance_after_rise = []

for imb in imbalances_after_fall :
    if not imb.is_full_filled :
        unfilled_imbalance_after_fall.append(imb)
        
for imb in imbalances_after_rise :
    if not imb.is_full_filled :
        unfilled_imbalance_after_rise.append(imb)
        
# Affiche l'emsemble des imbalances
all_unfilled_imbalances = []
all_unfilled_imbalances.extend(unfilled_imbalance_after_fall)
all_unfilled_imbalances.extend(unfilled_imbalance_after_rise)


In [ ]:
for imb in unfilled_imbalance_after_fall[-10:] : 
    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

Les scripts ci dessous ont pour but de récupérer le prix actuel du marché

In [ ]:
import nest_asyncio
import asyncio
import datetime
nest_asyncio.apply()

from binance import AsyncClient, BinanceSocketManager


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.trade_socket('BTCUSDT')
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            
            trade = Trade(res)
            print(trade.event_time, trade.event_type, trade.symbol, trade.price, trade.quantity)
            
    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Je reçois le dernier trade.
Quelles sont les informations que je reçois pour le trade ?
Quelles sont les informations que je reçois pour les bougies ?
Comment puis-je créer une bougie à partir des différents trades ?



In [ ]:
# Informations reçu pour les trades
{
  "e": "trade",     // Event type
  "E": 1672515782136,   // Event time
  "s": "BNBBTC",    // Symbol
  "t": 12345,       // Trade ID
  "p": "0.001",     // Price
  "q": "100",       // Quantity
  "b": 88,          // Buyer order ID
  "a": 50,          // Seller order ID
  "T": 1672515782136,   // Trade time
  "m": true,        // Is the buyer the market maker?
  "M": true         // Ignore
}

# Accéder aux différentes valeurs
            event_type = res["e"] # "trade"
            event_time = datetime.datetime.fromtimestamp(res['E'] / 1000, tz=datetime.timezone.utc) # 1672515782136
            event_time = event_time.astimezone(pytz.timezone('Europe/Paris'))
            symbol = res["s"] # "BNBBTC"
            trade_id = res["t"] # 12345
            price = res["p"] # "0.001"
            quantity = res["q"] # "100"
            buyer_order_id = res["b"] # 88
            selle_order_id = res["a"] # 50
            trade_time = datetime.datetime.fromtimestamp(res['T'] / 1000, tz=datetime.timezone.utc) # 1672515782136
            trade_time = trade_time.astimezone(pytz.timezone('Europe/Paris'))
            is_the_buyer_the_market_maker = res["m"] # true
            ignore = res["M"] # true

In [ ]:
# Informations reçu pour les bougies
timestamp=row['timestamp'],
open_price=row['open'],
high_price=row['high'],
low_price=row['low'],
close_price=row['close'],
volume=row['volume'],
close_time=row['close_time'],
quote_asset_volume=row['quote_asset_volume'],
number_of_trades=row['number_of_trades'],
taker_buy_base_asset_volume=row['taker_buy_base_asset_volume'],
taker_buy_quote_asset_volume=row['taker_buy_quote_asset_volume'],
ignore=row['ignore']

In [ ]:
trades = []


In [ ]:
'''# Importer la bibliothèque numpy pour les statistiques
import numpy as np

# Extraire les temps pour être rempli de la liste d'imbalances
time_to_be_filled = [imbalance.time_to_be_fullfilled for imbalance in fullfilled_imbalances_after_fall]
time_to_be_filled_seconds = [time.total_seconds() for time in time_to_be_filled]

# Calculer les statistiques
mean_time_to_fill = np.mean(time_to_be_filled)  # Moyenne
median_time_to_fill = np.median(time_to_be_filled)  # Médiane
min_time_to_fill = np.min(time_to_be_filled)  # Minimum
max_time_to_fill = np.max(time_to_be_filled)  # Maximum
std_time_to_fill = np.std(time_to_be_filled_seconds)  # Écart-type

# Calculer les percentiles
percentiles = np.percentile(time_to_be_filled, [25, 50, 75, 80, 85, 90, 95, 99])

# Afficher les statistiques
print("Mean Time to Fill: ", mean_time_to_fill)
print("Median Time to Fill: ", median_time_to_fill)
print("Min Time to Fill: ", min_time_to_fill)
print("Max Time to Fill: ", max_time_to_fill)
print("Standard Deviation of Time to Fill: ", std_time_to_fill)
print("25th Percentile: ", percentiles[0])
print("50th Percentile (Median): ", percentiles[1])
print("75th Percentile: ", percentiles[2])
print("80th Percentile: ", percentiles[3])
print("85th Percentile: ", percentiles[4])
print("90th Percentile: ", percentiles[5])
print("95th Percentile: ", percentiles[6])
print("99th Percentile: ", percentiles[7])'''

In [ ]:
'''# retourne une liste des jours où il y a eu des imbalances qui n'ont pas été comblés par la suite
random_imbalance_after_a_rise = find_imbalances_after_rise(df)

df_exp = pd.DataFrame(random_imbalance_after_a_rise, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df_exp['timestamp'] = pd.to_datetime(df_exp['timestamp'], unit='ms')
df_exp.set_index('timestamp', inplace=True)'''

In [ ]:
'''# code à modifier afin de pouvoir travailler avec tradingview sur des timeframes en heure et en minutes.
# Je suis limité à 5000 barres dans tradingview. Je dois donc réduire la liste des imbalances à ceux réalisés dans les 5000 derniers jours à partir d'aujourd'hui
# Get today's date
today = date.today()

# Calculate the date 5000 days ago
days_ago = today - timedelta(days=5000)
print(days_ago)

# Filter dates within the last 5000 days
recent_dates = [d for d in list_of_imbalance_start if d >= days_ago]

# Print recent dates
print("Recent dates within the last 5000 days:")
for d in recent_dates:
    print(d)'''